In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from astropy.modeling import models
from astropy import units as u
from astropy import nddata
from astropy.io import fits
import re
import os
from matplotlib.colors import LogNorm

In [5]:
from scipy import stats  
from scipy import ndimage
from datetime import datetime, timedelta
from mpl_toolkits.mplot3d import Axes3D
import scipy.signal

In [6]:
import matplotlib
matplotlib.rc('xtick', labelsize=20) 
matplotlib.rc('ytick', labelsize=20) 

In [7]:
# to enlarge the sizes
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (8, 8),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
plt.rcParams.update(params)

In [8]:
from PIL import Image

In [58]:
%%javascript
var kernel = IPython.notebook.kernel;
var thename = window.document.getElementById("notebook_name").innerHTML;
var command = "theNotebook = " + "'"+thename+"'";
kernel.execute(command);

<IPython.core.display.Javascript object>

In [59]:
print theNotebook

ViewIRImages


In [9]:
movies=['film_2h00.mov','film_2h25.mov','film_3h36.mov']

In [44]:
SelectTagRe='^image([0-9]+).tiff$' # regular expression to select the file
SearchTagRe='^image[0-9]+.tiff$'
MIN_IMGNUMBER=0
MAX_IMGNUMBER=9999
NBIMGPERLROW=4
SEL=0
object_name=movies[SEL]

In [13]:
NBMOVIES=len(movies)

In [14]:
input_dirs = [ mv.split('.')[0]  for mv in movies]

In [60]:
dir_top_images="images_notebooks/"+theNotebook ### Top directory to save images

In [45]:
inputdir = input_dirs[SEL]

In [16]:
input_filelist=os.listdir(inputdir)

In [26]:
indexes_files= [] # global list
filelist_tiffimages= []  # global list
dirlist_all = [inputdir]

In [27]:
def MakeFileList():
    """
    MakeFileList : Make The List of files to open
    =============
    
    - we select the files which are of interest.
    - In particular select the number range
    
    """
    count=0
    
    for dir in dirlist_all: # loop on directories, one per image   (if necessary)    
            dir_leaf= dir # build the name of leaf directory
            listfiles=os.listdir(dir_leaf) 
            for filename in listfiles:
                if re.search(SearchTagRe,filename):  #example of filename filter
                    str_index=re.findall(SelectTagRe,filename)
                    count=count+1
                    index=int(str_index[0])
                    if index >= MIN_IMGNUMBER and index <= MAX_IMGNUMBER: 
                        indexes_files.append(index)         
                        shortfilename=dir_leaf+'/'+filename
                        filelist_tiffimages.append(shortfilename)
                    
    return

In [28]:
MakeFileList()   # Make the list

In [31]:
indexes_files=np.array(indexes_files)
filelist_tiffimages=np.array(filelist_tiffimages)

In [32]:
sorted_indexes=np.argsort(indexes_files) # sort the file indexes
sorted_numbers=indexes_files[sorted_indexes]
sorted_files=filelist_tiffimages[sorted_indexes]

In [33]:
sorted_filenames=[]
for file in sorted_files:
    #fullfilename=os.path.join(inputdir,file)
    fullfilename=file
    sorted_filenames.append(fullfilename)

In [34]:
sorted_filenames=np.copy(sorted_files)

In [63]:
def ShowRawImages():
    """
    ShowRawImages: Show the raw images without background subtraction
    ==============
    """
    #global all_images
    #global all_titles
    #global object_name

    NBIMAGES=len(sorted_filenames)
    MAXIMGROW=int(NBIMAGES/NBIMGPERLROW)+1

    f, axarr = plt.subplots(MAXIMGROW,NBIMGPERLROW,figsize=(25,350))
    for index in np.arange(0,NBIMAGES):
        ix=index%NBIMGPERLROW
        iy=index/NBIMGPERLROW
        img = Image.open(sorted_filenames[index])
        iar_shp = np.array(img).shape
        #img = img.convert('L')  ## ? Convert gray scale
        # Load grayscale matrix, reshape to dimensions of color bmp
        iarr = np.array(img.getdata()).reshape(iar_shp[0], iar_shp[1])
        im=axarr[iy,ix].imshow(iarr,origin='lower',cmap='gray')
        axarr[iy,ix].set_title(str(sorted_numbers[index]))
        axarr[iy,ix].grid(color='white', ls='solid')
        #axarr[iy,ix].text(1000.,1700,all_filt[index],verticalalignment='bottom', horizontalalignment='center',color='yellow', fontweight='bold',fontsize=16)
    title='Raw images of {}'.format(object_name)
    plt.suptitle(title,size=16)   
    figfilename='cloudimg_{}'+object_name+'.pdf'
    figfullfilename=os.path.join(dir_top_images,figfilename)
    #plt.savefig(figfullfilename) VERY LONG NOT CONVENIENT

In [ ]:
ShowRawImages()

In [ ]:
#n, bins, patches=plt.hist(imarray.flatten(),bins=100,range=[1,1000],histtype='stepfilled')
#plt.grid()